## 필요한 라이브러리 설치


In [ ]:
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 GB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 71.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.1+cu116
    Uninstalling torchvision-0.14.1+cu116:
      Successfully uninstalled torchvision-0.14.1+cu116
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.13.1+cu116
    Uninstalling torchaudio-0.13.1+cu116:
      Successfully uninstalled torchaudio-0.13.1+cu116
ERROR: pip's dependency resolver does not currently take into account al

In [1]:
import pandas as pd
import os
import numpy as np
import shutil
import torch
from tqdm import tqdm

from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
print(torch.__version__)

torch.cuda.is_available()		#cuda 사용가능 여부 확인

1.13.1+cu116


True

In [3]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [2]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!pwd

/content/drive/MyDrive


In [ ]:
# 설치파일, 구글 드라이브에 깔아서 두번 안해도 안깔아도 됨. (진행완료)
!git clone https://github.com/ultralytics/yolov5.git

fatal: destination path 'yolov5' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
%pip install -r yolov5/yolov5/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 데이터 전처리

In [5]:
path = '/content/drive/MyDrive/detection/detection/'

In [ ]:
# 라벨 csv 파일 전처리 
def label_preclaer(df):
    width = 480
    height = 300
    
    new_df = df[['frame', 'class_id']]

    new_df['x_center'] = (df['xmin'] + df['xmax'])/2
    new_df['x_center'] = new_df['x_center'] / width
    
    new_df['y_center'] = (df['ymin'] + df['ymax'])/2
    new_df['y_center'] = new_df['y_center'] / height
    
    new_df['w'] = (df['xmax'] - df['xmin'])/width
    new_df['h'] = (df['ymax'] - df['ymin'])/height

    new_df['class_id'] = df['class_id'] - 1 

    return new_df[['frame', 'x_center', 'y_center', 'w', 'h', 'class_id']]

In [ ]:
train_labels = pd.read_csv(path + 'labels/labels_train.csv')
test_labels = pd.read_csv(path + 'labels/labels_val.csv')

In [ ]:
# 전처리 하기 전 데이터
train_labels.head(5)

,frame,xmin,xmax,ymin,ymax,class_id
0,1478019952686311006.jpg,237,251,143,155,1
1,1478019952686311006.jpg,437,454,120,186,3
2,1478019953180167674.jpg,218,231,146,158,1
3,1478019953689774621.jpg,171,182,141,154,2
4,1478019953689774621.jpg,179,191,144,155,1


In [ ]:
train_labels = label_preclaer(train_labels)
test_labels = label_preclaer(test_labels)

<ipython-input-9-cd185e654c28>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['x_center'] = (df['xmin'] + df['xmax'])/2
<ipython-input-9-cd185e654c28>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['x_center'] = new_df['x_center'] / width
<ipython-input-9-cd185e654c28>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [ ]:
# 전처리 하고 난 후 데이터
train_labels.head(5)

,frame,x_center,y_center,w,h,class_id
0,1478019952686311006.jpg,0.508333,0.496667,0.029167,0.040000,0
1,1478019952686311006.jpg,0.928125,0.510000,0.035417,0.220000,2
2,1478019953180167674.jpg,0.467708,0.506667,0.027083,0.040000,0
3,1478019953689774621.jpg,0.367708,0.491667,0.022917,0.043333,1
4,1478019953689774621.jpg,0.385417,0.498333,0.025000,0.036667,0


In [ ]:
label_unique = train_labels['frame'].unique()
label_unique2 = label_unique[:11000]
print(label_unique.shape)
print(label_unique2.shape)

test_labels_unique = test_labels['frame'].unique()
print(test_labels_unique.shape)

(18000,)
(11000,)
(4241,)


In [ ]:
# 총 22241 개 이미지 데이터
# -> 학습이 너무 느려서 train 18000 -> 11000로 줄임...

In [ ]:
# train, val 데이터 나누기
train_labels_unique, val_labels_unique = train_test_split(label_unique2, 
                                                          test_size=0.2,
                                                          random_state=42)

In [ ]:
train_labels_unique.shape, val_labels_unique.shape, test_labels_unique.shape

((8800,), (2200,), (4241,))

In [ ]:
# train : 8800, val : 2200, test : 4241 개 이미지 데이터.

In [ ]:
print(train_labels_unique[0])
print(val_labels_unique[0])
print(test_labels_unique[0])

1479499943071296595.jpg
1478020218191884276.jpg
1478899046136829030.jpg


In [ ]:
# 중복이 있는지 확인. 
for i in val_labels_unique:
    if i in train_labels_unique:
        print(i)
    else:
        pass
    
for i in train_labels_unique:
    if i in val_labels_unique:
        print(i)
    else:
        pass

In [ ]:
train_labels_df = train_labels[train_labels['frame'].isin(train_labels_unique)]
val_labels_df = train_labels[train_labels['frame'].isin(val_labels_unique)]
test_labels_df = test_labels.copy()

In [ ]:
train_labels_df.shape, val_labels_df.shape, test_labels_df.shape

((61162, 6), (14734, 6), (32699, 6))

In [ ]:
print(train_labels_df['frame'].head(5))
print(val_labels_df['frame'].head(5))
print(test_labels_df['frame'].head(5))

2    1478019953180167674.jpg
3    1478019953689774621.jpg
4    1478019953689774621.jpg
5    1478019953689774621.jpg
6    1478019953689774621.jpg
Name: frame, dtype: object
0     1478019952686311006.jpg
1     1478019952686311006.jpg
8     1478019954186238236.jpg
9     1478019954186238236.jpg
10    1478019954186238236.jpg
Name: frame, dtype: object
0    1478899046136829030.jpg
1    1478899046136829030.jpg
2    1478899046136829030.jpg
3    1478899046136829030.jpg
4    1478899046136829030.jpg
Name: frame, dtype: object


In [ ]:
# 중복이 있는지 확인. 
for i in val_labels_df['frame']:
    if i in train_labels_df['frame']:
        print(i)
    else:
        pass
    
for i in train_labels_df['frame']:
    if i in val_labels_df['frame']:
        print(i)
    else:
        pass

In [ ]:
# 리스트안 항목 개수 세어주는 함수 생성
def n_list(list):
    array = np.array(list)
    n = array.shape[0]

    return n

In [ ]:
filenames = os.listdir(path + 'images') 
filenames[0]

'1478896665001506986.jpg'

In [ ]:
n_file = n_list(filenames)
n_file

11

In [ ]:
yolo_path = '/content/drive/MyDrive/yolov5/yolov5/'

In [ ]:
# 폴더 생성하는 함수, 2번 실행 안해도 됨. (진행완료)
os.mkdir(yolo_path + 'dataset')

In [ ]:
# 폴더 생성하는 함수, 2번 실행 안해도 됨. (진행완료)
os.mkdir(yolo_path + 'dataset/images')
os.mkdir(yolo_path + 'dataset/labels')

In [ ]:
# 폴더 생성하는 함수, 2번 실행 안해도 됨. (진행완료)
os.mkdir(yolo_path + 'dataset/images/train')
os.mkdir(yolo_path + 'dataset/images/val')
os.mkdir(yolo_path + 'dataset/images/test')

In [ ]:
# 라벨링 폴더 구분. 2번 실행 안해도 됨. (진행완료)
os.mkdir(yolo_path + 'dataset/labels/train')
os.mkdir(yolo_path + 'dataset/labels/val')
os.mkdir(yolo_path + 'dataset/labels/test')

In [ ]:
print(path)
print(yolo_path)

/content/drive/MyDrive/detection/detection/
/content/drive/MyDrive/yolov5/yolov5/


In [ ]:
train_filenames = os.listdir(path + 'images/train')
val_filenames = os.listdir(path + 'images/val')
test_filenames = os.listdir(path + 'images/test')

In [ ]:
# 이미지 파일 train, val, test 나누기. 2번 실행 안해도 됨. (진행완료)
for file_name in train_labels_unique: 
    if file_name in train_filenames:
        shutil.copy(path + 'images/train/' + file_name, 
                    yolo_path + '/dataset/images/train/' + file_name)
    else: 
        pass

for file_name in val_labels_unique: 
    if file_name in val_filenames:
        shutil.copy(path + 'images/val/' + file_name, 
                    yolo_path + 'dataset/images/val/' + file_name)
    else:
        pass

for file_name in test_labels_unique: 
    if file_name in test_filenames:
        shutil.copy(path + 'images/test/' + file_name, 
                    yolo_path + 'dataset/images/test/' + file_name)
    else:
        pass

In [ ]:
train_filenames = os.listdir(yolo_path + 'dataset/images/train') 
val_filenames = os.listdir(yolo_path + 'dataset/images/val') 
test_filenames = os.listdir(yolo_path + 'dataset/images/test') 

In [ ]:
n_train = n_list(train_filenames)
n_val = n_list(val_filenames)
n_test = n_list(test_filenames)

n_train, n_val, n_test

(14400, 3600, 4241)

## 라벨링 csv -> txt 변경

In [ ]:
train_labels_df['filename'] = train_labels_df['frame'].apply(lambda x: x.split('.')[0])
val_labels_df['filename'] = val_labels_df['frame'].apply(lambda x: x.split('.')[0])
test_labels_df['filename'] = test_labels_df['frame'].apply(lambda x: x.split('.')[0])

<ipython-input-79-7529b86085b0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_labels_df['filename'] = train_labels_df['frame'].apply(lambda x: x.split('.')[0])
<ipython-input-79-7529b86085b0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_labels_df['filename'] = val_labels_df['frame'].apply(lambda x: x.split('.')[0])


In [ ]:
train_labels_df['filename'].head(5)

2    1478019953180167674
3    1478019953689774621
4    1478019953689774621
5    1478019953689774621
6    1478019953689774621
Name: filename, dtype: object

In [ ]:
train_labels_df.head(1)

,frame,x_center,y_center,w,h,class_id,filename
2,1478019953180167674.jpg,0.467708,0.506667,0.027083,0.04,0,1478019953180167674


In [ ]:
# train 라벨링 csv -> txt 파일로 변경하는 작업 한번만 진행 (진행완료)
for name, mini in tqdm(train_labels_df.groupby('filename')):
    with open(yolo_path + 'dataset/labels/train/' + name + '.txt', 'w+') as f:
        row = mini[['class_id', 'x_center', 'y_center', 'w', 'h']].astype(float).values
        row = row.astype(str)
        for j in range(len(row)):
            text = ' '.join(row[j])
            f.write(text)
            f.write("\n")

100%|██████████| 8800/8800 [01:03<00:00, 137.83it/s]


In [ ]:
# val 라벨링 csv -> txt 파일로 변경하는 작업 한번만 진행 (진행완료)
for name, mini in tqdm(val_labels_df.groupby('filename')):
    with open(yolo_path + 'dataset/labels/val/' + name + '.txt', 'w+') as f:
        row = mini[['class_id', 'x_center', 'y_center', 'w', 'h']].astype(float).values
        row = row.astype(str)
        for j in range(len(row)):
            text = ' '.join(row[j])
            f.write(text)
            f.write("\n")

100%|██████████| 2200/2200 [00:14<00:00, 149.38it/s]


In [ ]:
# test 라벨링 csv -> txt 파일로 변경하는 작업 한번만 진행 (진행완료)
for name, mini in tqdm(test_labels_df.groupby('filename')):
    with open(yolo_path + 'dataset/labels/test/' + name + '.txt', 'w+') as f:
        row = mini[['class_id', 'x_center', 'y_center', 'w', 'h']].astype(float).values
        row = row.astype(str)
        for j in range(len(row)):
            text = ' '.join(row[j])
            f.write(text)
            f.write("\n")

100%|██████████| 4241/4241 [00:28<00:00, 149.33it/s]


## 모델 YOLOv5 수정

In [ ]:
# 모델 yolov5s 
# yolov5s : nc 5로 수정 (수동으로 바꿈.)

In [4]:
!pwd

/content/drive/MyDrive


In [ ]:
# yolov5 데이터 파일 생성. 두번 실행 안해도 됨 (진행완료)
with open(yolo_path + '/dataset/data.yaml', 'w+') as f:
    f.write('train: "/content/drive/MyDrive/yolov5/yolov5/dataset/images/train" \n'
    'val: "/content/drive/MyDrive/yolov5/yolov5/dataset/images/val"\n'
    '\n'
    'nc: 5\n'
    'names: [\n'
    '"car",\n'
    '"truck",\n'
    '"pedestrian",\n'
    '"bicyclist",\n'
    '"light",\n'
    ']')

In [4]:
%cd /content/drive/MyDrive/yolov5/yolov5/

/content/drive/MyDrive/yolov5/yolov5


In [7]:
# 모델 학습 
!python train.py --data dataset/data.yaml --cfg models/yolov5s.yaml --weights yolov5s.pt --batch 64 --epochs 30 --name detection

train: weights=yolov5s.pt, cfg=models/yolov5s.yaml, data=dataset/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=detection, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 3 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 v7.0-117-g85f6019 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, ob

In [9]:
# 검증하기      
!python val.py --data dataset/data.yaml --img 480 --weights /content/drive/MyDrive/yolov5/yolov5/runs/train/detection5/weights/best.pt

val: data=dataset/data.yaml, weights=['/content/drive/MyDrive/yolov5/yolov5/runs/train/detection5/weights/best.pt'], batch_size=32, imgsz=480, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-117-g85f6019 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/drive/MyDrive/yolov5/yolov5/dataset/labels/val.cache... 672 images, 2927 backgrounds, 1 corrupt: 100% 3600/3600 [00:00<?, ?it/s]
val: WARNING ⚠️ /content/drive/MyDrive/yolov5/yolov5/dataset/images/val/1478020832210646675.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/drive/MyDrive/yolov5/yolov5/dataset/images/val/1478732887556135464.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /

In [ ]:
# 최종 테스트 셋에 대한 성능 지표 
!python val.py --data dataset/data.yaml --img 480 --weights /content/drive/MyDrive/yolov5/yolov5/runs/train/detection5/weights/best.pt

val: data=dataset/data.yaml, weights=['/content/drive/MyDrive/yolov5/yolov5/runs/train/detection5/weights/best.pt'], batch_size=32, imgsz=480, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-117-g85f6019 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/drive/MyDrive/yolov5/yolov5/dataset/labels/test... 3254 images, 0 backgrounds, 0 corrupt:  77% 3254/4241 [14:38<04:10,  3.94it/s]

In [ ]:
!python detect.py --weights '/content/drive/MyDrive/yolov5/yolov5/runs/train/detection5/weights/best.pt' --source /content/drive/MyDrive/yolov5/yolov5/dataset/images/test

detect: weights=['/content/drive/MyDrive/yolov5/yolov5/runs/train/detection2/weights/best.pt'], source=/content/drive/MyDrive/yolov5/yolov5/dataset/images/test, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-117-g85f6019 Python-3.9.16 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
image 1/4241 /content/drive/MyDrive/yolov5/yolov5/dataset/images/test/1478899046136829030.jpg: 416x640 3 cars, 17.2ms
image 2/4241 /content/drive/MyDrive/yolov5/yolov5/dataset/images/test/1478899046707290405.jpg: 416x640 3 cars, 11.3ms
image 3/4241 /content/drive

In [ ]:
!zip -r /content/drive/MyDrive/yolov5.zip /content/drive/MyDrive/yolov5/yolov5

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  adding: content/drive/MyDrive/yolov5/yolov5/dataset/labels/test/1478901254171487354.txt (deflated 69%)
  adding: content/drive/MyDrive/yolov5/yolov5/dataset/labels/test/1478901254743514940.txt (deflated 68%)
  adding: content/drive/MyDrive/yolov5/yolov5/dataset/labels/test/1478901255313775668.txt (deflated 66%)
  adding: content/drive/MyDrive/yolov5/yolov5/dataset/labels/test/1478901255885676310.txt (deflated 63%)
  adding: content/drive/MyDrive/yolov5/yolov5/dataset/labels/test/1478901256457621484.txt (deflated 66%)
  adding: content/drive/MyDrive/yolov5/yolov5/dataset/labels/test/1478901257027780352.txt (deflated 58%)
  adding: content/drive/MyDrive/yolov5/yolov5/dataset/labels/test/1478901257599002347.txt (deflated 63%)
  adding: content/drive/MyDrive/yolov5/yolov5/dataset/labels/test/1478901258170371023.txt (deflated 68%)
  adding: content/drive/MyDrive/yolov5/yolov5/dataset/labels/test/1478901258742151627.txt (deflated 68%)
  adding: content/d